In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=b1e28eeedd0e5fe352d65b3079e7b4e354b12abc7c07ae5d65fea820e773b8ff
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np


In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [5]:
model_name = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters : {trainable_model_params}\n all model parameters: {all_model_params} \n percentage : {trainable_model_params/all_model_params}"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters : 247577856
 all model parameters: 247577856 
 percentage : 1.0


In [66]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation. \n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors='pt').input_ids

    return example

tokenized_datasets = dataset.map(tokenize_function,batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary'])

In [67]:
# tokenized_datasets = tokenized_datasets.filter(lambda example, index : index %3 ==0 , with_indices=True)

In [68]:
print(f"Shapes of datasets: ")
print(f"Training {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of datasets: 
Training (12460, 2)
Validation: (500, 2)
Test: (1500, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})


In [10]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate=1e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=100
)

trainer = Trainer(
    model = original_model,
    args = training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer.train()

Step,Training Loss
1,49.000000
2,50.500000
3,49.000000
4,49.250000
5,49.000000
6,49.500000
7,50.000000
8,48.750000
9,49.250000
10,49.250000


TrainOutput(global_step=100, training_loss=47.865, metrics={'train_runtime': 285.0113, 'train_samples_per_second': 2.807, 'train_steps_per_second': 0.351, 'total_flos': 535480249614336.0, 'train_loss': 47.865, 'epoch': 6.25})

In [12]:
rouge = evaluate.load('rouge')

In [13]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []

for _, dialogue in enumerate(dialogues):
  prompt = f"""
  Summarize the following conversation.
  {dialogue}
  Summary:

  """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

  original_model_outputs = original_model.generate(input_ids = input_ids, generation_config=GenerationConfig(max_new_tokens=200,num_beams=1))
  original_model_text_outputs = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
  original_model_summaries.append(original_model_text_outputs)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries))




df = pd.DataFrame(zipped_summaries, columns=['human','original'])


In [14]:
df.head()

,human,original
0,Ms. Dawson helps #Person1# to write a memo to ...,This memo is intended to be a dictation of all...
1,In order to prevent employees from wasting tim...,The memo is being prepared for the intra-offic...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo should be distributed to all employee...
3,#Person2# arrives late because of traffic jam....,The person who's been stuck in traffic is tryi...
4,#Person2# decides to follow #Person1#'s sugges...,The person who is driving to work is a driver.


In [15]:
original_model_results = rouge.compute(

  predictions=original_model_summaries,
  references=human_baseline_summaries[0:len(original_model_summaries)],
  use_aggregator=True,
  use_stemmer=True
)

In [16]:
original_model_results

{'rouge1': 0.27021327650852306,
 'rouge2': 0.08796296296296295,
 'rougeL': 0.23658824694060435,
 'rougeLsum': 0.23770935749066424}

In [17]:
%pip install loralib \
peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 19.0 MB/s eta 0:00:00


In [98]:
model_name = "google/flan-t5-base"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [99]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r =  40,
    lora_alpha = 40,
    target_modules=["q","v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [100]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters : 4423680
 all model parameters: 252001536 
 percentage : 0.017554178717386865


In [101]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
   output_dir='./results',
   save_steps=999999999,
   save_total_limit=None,
    auto_find_batch_size=True,
    learning_rate = 1e-3,
    save_strategy="no",      # Prevents saving final model

    num_train_epochs=1,
   logging_steps=1,
    max_steps=300
)



peft_trainer = Trainer(
    model=peft_model,
    args = peft_training_args,
    train_dataset=tokenized_datasets["train"]
)

max_steps is given, it will override any value given in num_train_epochs


In [102]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

In [103]:
peft_trainer.train()

peft_model_path='./peft-dialogue-summary-checkpoint-local'
tokenizer.save_pretrained(peft_model_path)
peft_trainer.model.save_pretrained(peft_model_path)
# peft_trainer.save_model(output_path=peft_model_path)


Step,Training Loss
1,49.250000
2,45.750000
3,42.500000
4,37.000000
5,32.250000
6,28.000000
7,26.625000
8,23.250000
9,20.125000
10,18.125000


In [104]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

peft_model_summaries = []

for _, dialogue in enumerate(dialogues):
  prompt = f"""
  Summarize the following conversation.
  {dialogue}
  Summary:

  """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

  peft_model_outputs = peft_model.generate(input_ids = input_ids, generation_config=GenerationConfig(max_new_tokens=200,num_beams=1))
  peft_model_text_outputs = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
  peft_model_summaries.append(peft_model_text_outputs)

zipped_summaries = list(zip(human_baseline_summaries, peft_model_summaries))




df = pd.DataFrame(zipped_summaries, columns=['human','original'])


In [105]:
df

,human,original
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person2# asks #Person2# to take a dictation f...
1,In order to prevent employees from wasting tim...,#Person1# asks #Person2# to take a dictation f...
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1# tells #Person2# #Person2# to take a ...
3,#Person2# arrives late because of traffic jam....,#Person1# tells #Person2# how to avoid congest...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1# got stuck in traffic. #Person2# thin...
5,#Person2# complains to #Person1# about the tra...,#Person1# thinks it's better to take public tr...
6,#Person1# tells Kate that Masha and Hero get d...,#Person1## tells #Person2# that Masha and Hero...
7,#Person1# tells Kate that Masha and Hero are g...,#Person1# is getting divorced from #Person2#. ...
8,#Person1# and Kate talk about the divorce betw...,#Person1# tells #Person2# that Masha and Hero ...
9,#Person1# and Brian are at the birthday party ...,#Person2# asks Brian to dance with #Person1#. ...


In [106]:
peft_model_results = rouge.compute(

  predictions=peft_model_summaries,
  references=human_baseline_summaries,
  use_aggregator=True,
  use_stemmer=True
)

In [107]:
peft_model_results

{'rouge1': 0.2953008662871226,
 'rouge2': 0.09706963412020822,
 'rougeL': 0.2287152933419754,
 'rougeLsum': 0.23016209424904566}

In [108]:
human_base_line_train =  dataset['train'][0:10]['summary']
dialogues = dataset['train'][0:10]['dialogue']
peft_model_summaries_train = []

for _, dialogue in enumerate(dialogues):
  prompt = f"""
  Summarize the following conversation.
  {dialogue}
  Summary:

  """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

  peft_model_outputs = peft_model.generate(input_ids = input_ids, generation_config=GenerationConfig(max_new_tokens=200,num_beams=1))
  peft_model_text_outputs = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
  peft_model_summaries_train.append(peft_model_text_outputs)

zipped_summaries = list(zip(human_baseline_summaries, peft_model_summaries))

In [109]:
peft_model_results = rouge.compute(

  predictions=peft_model_summaries_train,
  references=human_base_line_train,
  use_aggregator=True,
  use_stemmer=True
)

In [110]:
peft_model_results

{'rouge1': 0.24470476258823526,
 'rouge2': 0.08559464615192171,
 'rougeL': 0.19063505771412295,
 'rougeLsum': 0.19025184431128625}